<div style="text-align:center;font-size:22pt; font-weight:bold;color:white;border:solid black 1.5pt;background-color:#1e7263;">
    Understanding Model History Object: Classification Task
</div>

In [1]:
# ======================================================================= #
# Course: Deep Learning Complete Course (CS-501)
# Author: Dr. Saad Laouadi
# Institution: Quant Coding Versity Academy
# Date: December 25, 2024
#
# ==========================================================
# Lesson: Understanding Model History Object in Keras
#         Analyzing and Visualizing Training Progress
# ==========================================================
# ## Learning Objectives
# This guide will enable you to:
# 1. Access and interpret the model.fit() history object
# 2. Extract and analyze training metrics over epochs
# 3. Visualize training and validation metrics
# 4. Identify optimal training epochs and model performance
# 5. Detect overfitting through history analysis
# =======================================================================
#          Copyright © Dr. Saad Laouadi 2024
# =======================================================================

In [2]:
# ==================================================== #
#        Load Required Libraries
# ==================================================== #

import os  
import shutil
from datetime import datetime

# Disable Metal API Validation
os.environ["METAL_DEVICE_WRAPPER_TYPE"] = "0"  


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_covtype
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import (
    ModelCheckpoint, 
    EarlyStopping, 
    ReduceLROnPlateau, 
    CSVLogger
)

from tensorflow.keras.utils import to_categorical

from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_curve,
    auc,
    precision_recall_curve
)


# import tensorflow
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Set styling for better visualization
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("="*72)

%reload_ext watermark
%watermark -a "Dr. Saad Laouadi" -u -d -m

print("="*72)
print("Imported Packages and Their Versions:")
print("="*72)

%watermark -iv
print("="*72)

# Global Config
RANDOM_STATE = 101

Author: Dr. Saad Laouadi

Last updated: 2025-01-04

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.1.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Imported Packages and Their Versions:
tensorflow: 2.16.2
matplotlib: 3.9.2
pandas    : 2.2.2
numpy     : 1.26.4
keras     : 3.6.0
sklearn   : 1.5.1
seaborn   : 0.13.2



In [3]:
def cleanup_directory(directory_path):
    """
    Deletes the specified directory and all its contents.

    Args:
        directory_path (str): Path to the directory to delete.
    """
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        shutil.rmtree(directory_path)
        print(f"Directory '{os.path.basename(directory_path)}' deleted successfully.")
    else:
        print(f"Directory '{os.path.basename(directory_path)}' does not exist or is not a directory.")

In [4]:
data = fetch_covtype()
print(data.DESCR)

.. _covtype_dataset:

Forest covertypes
-----------------

The samples in this dataset correspond to 30×30m patches of forest in the US,
collected for the task of predicting each patch's cover type,
i.e. the dominant species of tree.
There are seven covertypes, making this a multiclass classification problem.
Each sample has 54 features, described on the
`dataset's homepage <https://archive.ics.uci.edu/ml/datasets/Covertype>`__.
Some of the features are boolean indicators,
while others are discrete or continuous measurements.

**Data Set Characteristics:**

=================   ============
Classes                        7
Samples total             581012
Dimensionality                54
Features                     int
=================   ============

:func:`sklearn.datasets.fetch_covtype` will load the covertype dataset;
it returns a dictionary-like 'Bunch' object
with the feature matrix in the ``data`` member
and the target values in ``target``. If optional argument 'as_frame' is
se

In [5]:
data.feature_names

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area_0',
 'Wilderness_Area_1',
 'Wilderness_Area_2',
 'Wilderness_Area_3',
 'Soil_Type_0',
 'Soil_Type_1',
 'Soil_Type_2',
 'Soil_Type_3',
 'Soil_Type_4',
 'Soil_Type_5',
 'Soil_Type_6',
 'Soil_Type_7',
 'Soil_Type_8',
 'Soil_Type_9',
 'Soil_Type_10',
 'Soil_Type_11',
 'Soil_Type_12',
 'Soil_Type_13',
 'Soil_Type_14',
 'Soil_Type_15',
 'Soil_Type_16',
 'Soil_Type_17',
 'Soil_Type_18',
 'Soil_Type_19',
 'Soil_Type_20',
 'Soil_Type_21',
 'Soil_Type_22',
 'Soil_Type_23',
 'Soil_Type_24',
 'Soil_Type_25',
 'Soil_Type_26',
 'Soil_Type_27',
 'Soil_Type_28',
 'Soil_Type_29',
 'Soil_Type_30',
 'Soil_Type_31',
 'Soil_Type_32',
 'Soil_Type_33',
 'Soil_Type_34',
 'Soil_Type_35',
 'Soil_Type_36',
 'Soil_Type_37',
 'Soil_Type_38',
 'Soil_Type_39']

### ML Model Raodmap
1. Read the data
2. Explore the data

3. Processing
    - Numerical features (scaling)
    
4. Splitting the data

In [6]:
# ==================================================== #
#        Implementing ModelCheckpoint 
#        Callback with CLOUD Cover Dataset
# ==================================================== #

In [9]:
# Load the dataset
def load_and_preprocess_data(test_size=0.2):
    """
    Load and preprocess the Forest Cover Type dataset with memory efficient handling
    """
    print("Loading Cover Type dataset...")
    data = fetch_covtype()
    X = data.data
    y = data.target - 1  # Adjust classes to be 0-based
    
    print(f"Dataset shape: {X.shape}")
    print(f"Number of classes: {len(np.unique(y))}")
    
    # Scale features
    print("Scaling features...")
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Convert to categorical (one-hot encoding)
    print("Converting targets to one-hot encoding...")
    y_categorical = to_categorical(y)
    
    # Split the data
    print("Splitting dataset...")
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y_categorical,
        test_size=test_size,
        random_state=42,
        stratify=y  # Maintain class distribution
    )
    
    return X_train, X_test, y_train, y_test


def create_model(input_shape, num_classes):
    """
    Create a model architecture suitable for the Cover Type dataset
    """
    model = Sequential([
        tf.keras.layers.Input(shape = input_shape),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        # Hidden layers
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        
        # Output layer
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model


def setup_callbacks(experiment_name):
    """
    Setup training callbacks with logging
    """
    # Create directories for checkpoints and logs
    base_dir = f'covtype_training_{experiment_name}'
    checkpoint_dir = os.path.join(base_dir, 'checkpoints')
    logs_dir = os.path.join(base_dir, 'logs')
    
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(logs_dir, exist_ok=True)
    
    callbacks = [
        # Model checkpoint to save best models
        ModelCheckpoint(
            filepath=os.path.join(checkpoint_dir, 'model_best_val_acc.keras'),
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        ),
        
        # Another checkpoint to save best models by loss
        ModelCheckpoint(
            filepath=os.path.join(checkpoint_dir, 'model_best_val_loss.keras'),
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            verbose=1
        ),
        
        # Early stopping to prevent overfitting
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        ),
        
        # Reduce learning rate when training plateaus
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-6,
            verbose=1
        ),
        
        # CSV Logger to track all metrics
        CSVLogger(
            os.path.join(logs_dir, 'training_log.csv'),
            append=True
        )
    ]
    
    return callbacks


def train_model():
    """
    Main training function
    """
    # Load and preprocess data
    X_train, X_test, y_train, y_test = load_and_preprocess_data()
    
    # Create model
    num_classes = y_train.shape[1]
    model = create_model((X_train.shape[1],), num_classes)
    
    # Setup callbacks
    experiment_name = datetime.now().strftime("%Y%m%d_%H%M%S")
    callbacks = setup_callbacks(experiment_name)
    
    # Train model
    print("\nStarting model training...")
    history = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=50,
        batch_size=1024,            # Large batch size for efficiency
        callbacks=callbacks,
        verbose=1
    )
    
    # Evaluate on test set
    print("\nEvaluating model on test set...")
    test_results = model.evaluate(X_test, y_test, verbose=1)
    print(f"\nTest set results:")
    for metric_name, value in zip(model.metrics_names, test_results):
        print(f"{metric_name}: {value:.4f}")
    
    return model, history


def analyze_training_results(history, experiment_name):
    """
    Analyze and save training history
    """
    # Convert history to dataframe
    history_df = pd.DataFrame(history.history)
    
    # Save history to csv
    results_dir = f'covtype_training_{experiment_name}/analysis'
    os.makedirs(results_dir, exist_ok=True)
    history_df.to_csv(os.path.join(results_dir, 'training_history.csv'))
    
    # Print summary statistics
    print("\nTraining Summary:")
    print(f"Best validation accuracy: {max(history.history['val_accuracy']):.4f}")
    print(f"Best validation loss: {min(history.history['val_loss']):.4f}")
    print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")
    print(f"Final validation loss: {history.history['val_loss'][-1]:.4f}")

if __name__ == "__main__":
    model, history = train_model()
    analyze_training_results(history, datetime.now().strftime("%Y%m%d_%H%M%S"))

Loading Cover Type dataset...
Dataset shape: (581012, 54)
Number of classes: 7
Scaling features...
Converting targets to one-hot encoding...
Splitting dataset...

Starting model training...
Epoch 1/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6017 - loss: 1.1802
Epoch 1: val_accuracy improved from -inf to 0.74354, saving model to covtype_training_20241231_223140/checkpoints/model_best_val_acc.keras

Epoch 1: val_loss improved from inf to 0.60940, saving model to covtype_training_20241231_223140/checkpoints/model_best_val_loss.keras
364/364 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.6019 - loss: 1.1793 - val_accuracy: 0.7435 - val_loss: 0.6094 - learning_rate: 0.0010
Epoch 2/50
364/364 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7305 - loss: 0.6472
Epoch 2: val_accuracy improved from 0.74354 to 0.77084, saving model to covtype_training_20241231_223140/checkpoints/model_best_val_acc.keras

Epoch 2: val_loss improved from 0.60940 to 0.53573, saving model to covtype_

In [10]:
# Cleanup 
# cleanup_directory(checkpoint_dir)